<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [1]:
#| echo: false
#| output: asis
show_doc(Value)

---

### Value

>      Value ()

Initialize self.  See help(type(self)) for accurate signature.